In [2]:
from pdf2image import convert_from_path
from numpy import asarray
import numpy as np
import cv2
import pytesseract
from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
import pandas as pd

In [4]:
pdf_path = r"C:\Users\u\Desktop\python practice\CG_0a2898ab-ed08-4029-b860-b8c470af27d7_1777792 (2).pdf"
poppler_path = r"C:\Users\u\Downloads\poppler-22.04.0\Library\bin"

In [6]:
 # convert pdf to image
img = convert_from_path(pdf_path,fmt = 'png',poppler_path = poppler_path)

In [7]:
img

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1654x2339 at 0x188FD45ED00>]

In [12]:
 # Adjust image pixel for better ocr results
def adjust_gamma(image,gamma = 1.0):
        
        invGamma = 1.0/gamma
        table = np.array([((i/255)** invGamma)*255
                           for i in np.arange(0,256)]).astype('uint8')
        
        return cv2.LUT(image,table)
        
img_file =  asarray(img[0])
gamma = .5
    
adjusted = adjust_gamma(img_file,gamma = gamma)

In [14]:
custom_config = r'--oem 3 --psm 6'
text_data = pytesseract.image_to_data(adjusted,config=custom_config,
                                          output_type=Output.DICT)  

In [15]:
x= pytesseract.image_to_string(adjusted)

In [16]:
x

'Invoice\n\nPage 1 of 1\n\n \n\nBill-to Customer No. na\n\nPrices Including VAT No\nDocument Date 14. December 2020\nSalesperson Niall Gannon\n\n \n\nYour Order No.\nJob Location\nCheque No.\n\nOur Order No. |\n\nPayment Method ON A/C TRA\n\nGoods Collected By\n\n \n\n \n\nDelivery Instructions:\n\nUnit of VAT\nNo. Description Qty Measure Unit Price Discount % Identifier Amount\nFO27103 FIBERGLASS LADDER 3 STEP 3 EACH 35.00 HIGHC 105.00\nFO27104 FIBERGLASS LADDER 4 STEP 3 EACH 41.50 HIGHC 124.50\nFO27105 FIBERGLASS LADDER 5 STEP 3 EACH 47.00 HIGHC 141.00\nFO27106 FIBERGLASS LADDER 6 STEP 3 EACH 55.00 HIGHC 165.00\nLFD66AL ALUMINIUM 2 STEP STOOL 3 EACH 16.00 HIGHC 48.00\nLAD430GFA FIBERGLASS EXT LADDER 28STEP 7.6MTR 1 EACH 125.00 HIGHC 125.00\nLADS60GFA FIBERGLASS EXT LADDER 36STEP 9.7MTR 1 EACH 150.00 HIGHC 150.00\nTotal EUR Excl. VAT 858.50\n21% VAT 180.29\nTotal EUR Incl. VAT 1,038.79\n\nGoods remain strictly the property of the Vendor until paid in full\n\x0c'

In [19]:
df = pd.DataFrame(text_data)

In [20]:
df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,1654,2339,-1,
1,2,1,1,0,0,0,108,76,1437,2149,-1,
2,3,1,1,1,0,0,108,76,1437,2149,-1,
3,4,1,1,1,1,0,1141,76,403,55,-1,
4,5,1,1,1,1,1,1141,90,112,25,96.547035,Invoice
...,...,...,...,...,...,...,...,...,...,...,...,...
177,5,1,1,1,26,8,890,2199,90,21,96.666229,Vendor
178,5,1,1,1,26,9,989,2199,47,21,96.839722,until
179,5,1,1,1,26,10,1047,2199,49,26,96.737946,paid
180,5,1,1,1,26,11,1108,2199,18,20,96.874809,in


In [21]:
df = df.drop(["level","page_num","par_num","block_num","conf",
                  "width","height"],axis = 1)

In [22]:
def invoice_data_extraction(pdf_path,poppler_path):
    # pdf file location
    pdf_path = pdf_path
    # convert pdf to image
    img = convert_from_path(pdf_path,fmt = 'png',poppler_path = poppler_path)
    # Adjust image pixel for better ocr results
    def adjust_gamma(image,gamma = 1.0):
        
        invGamma = 1.0/gamma
        table = np.array([((i/255)** invGamma)*255
                           for i in np.arange(0,256)]).astype('uint8')
        
        return cv2.LUT(image,table)
        
    img_file =  asarray(img[0])
    gamma = .5

In [51]:
df = pd.DataFrame(text_data)
    
df = df.drop(["level","page_num","par_num","block_num","conf",
                  "width","height"],axis = 1)
Quantity = []
vat_price =[]
for idx,word in enumerate(df["text"]):
        if word.lower() == "Qty".lower() or word.lower() == "Quantity".lower():
             line_no_1 = df["line_num"][idx]
             new_df = df[idx:]
             new_df.reset_index(drop=True, inplace=True)
             position = df["left"][idx]
             #text = new_df[ new_df["word_num"] == word_no]
             # Extract quantity of each line items
             for t,(txt, j) in enumerate(zip(new_df["text"],new_df["left"])):
                if (abs(j-position) < 50) :
                    if txt.isnumeric() == True:
                         top_1 = new_df["top"][t]
                         
                         Quantity.append(int(txt))
                         
             # Extract Vat of each quantity  
             for i,(txt, k) in enumerate(zip(new_df["text"],new_df["left"])):
                if txt.lower() == "VAT".lower():
                     line_no_2 = new_df["line_num"][i]
                     
                     if line_no_1 == line_no_2:
                         vat = []
                         position_1 = new_df["left"][i]
                         for j,(txt, k) in enumerate(zip(new_df["text"],new_df["left"])):
                            if (abs(k-position_1) < 50) :
                                 top = new_df["top"][j]
                                
                                 if top <= (top_1+10):
                                    if (txt.replace('.', '', 1).isdigit()) == True:
                                     vat.append(txt)
                                      
                         vat_price.append(vat)            
        # extract total invoce amout and vat amount          
        elif word.lower() == "Total".lower():
            line_no = df["line_num"][idx]
            total_df = df[df["line_num"] == line_no]
            total_amount = ' '.join(total_df["text"])
            vat_line_df = df[df["line_num"] == (line_no-1)]
            total_vat = ' '.join(vat_line_df["text"])